In [11]:
#Just the basics to code sql
import psycopg2   as g2        #psyco must run with sqlalchemy
import sqlalchemy as sal       #needed for solid connection to postgres databases
import pandas     as pd        #needed for prettier sql result outputs
print("psycopg2:", g2.__version__)
print("sqlalchemy:", sal.__version__)
print("pandas:", pd.__version__)

psycopg2: 2.9.10 (dt dec pq3 ext lo64)
sqlalchemy: 2.0.38
pandas: 2.2.3


In [12]:
#Connect to psql w/ sal and g2 Note: Grok 3 says  ,text  is version related
from sqlalchemy import create_engine, text
user = "postgres"
password = "password"
host = "localhost"
port = "5433"
name = "rt"
pg = "postgresql"
#Create a string for the engine
c = f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{name}"
#Create engine
eg = create_engine(c)
#Test Connection
try:
    with eg.connect() as cn:
        result = cn.execute (text("SELECT version();"))
        print("BINGO!", result.fetchone()[0])
except Exception as e:
   print("Get to the Chawpaw!:", e)


BINGO! PostgreSQL 17.4 on x86_64-windows, compiled by msvc-19.42.34436, 64-bit


In [13]:
#Query SQL Tables
from sqlalchemy import text

query = "SELECT * FROM films LIMIT 5;"
df = pd.read_sql(text(query),eg)
df

,id,title,release_year,country,duration,language,certification,gross,budget
0,2,Over the Hill to the Poorhouse,1920.0,USA,110.0,,,3000000.0,100000.0
1,3,The Big Parade,1925.0,USA,151.0,,Not Rated,NaN,245000.0
2,4,Metropolis,1927.0,Germany,145.0,German,Not Rated,26435.0,6000000.0
3,5,Pandora's Box,1929.0,Germany,110.0,German,Not Rated,9950.0,NaN
4,6,The Broadway Melody,1929.0,USA,100.0,English,Passed,2808000.0,379000.0


In [14]:
###############################################        Introducing Joins Section 1 & 2          ####################################################
# NOTES
## All sql code has to be in red or else something is wrong
## When using, USING () are needed
## INNER JOINS returns exact matches only in each table
## LEFT JOIN allows nulls in right table; right join allows nulls on the left, full join allows nulls in either table
## When using GROUP BY the group must be in SELECT, duh.
## IS NULL in WHERE must have a field; ie name ...  OR name IS NULL

In [15]:
from sqlalchemy import text

query = "\
SELECT prime_ministers.country, prime_ministers.continent, prime_minister, president \
FROM presidents \
INNER JOIN prime_ministers \
ON presidents.country = prime_ministers.country \
LIMIT 5"
df = pd.read_sql(text(query),eg)
df

,country,continent,prime_minister,president
0,Egypt,Africa,Mostafa Madbouly,Abdel Fattah el-Sisi
1,Portugal,Europe,António Costa,Marcelo Rebelo de Sousa
2,Pakistan,Asia,Shehbaz Sharif,Arif Alvi
3,India,Asia,Narendra Modi,Ram Nath Kovind


In [16]:
from sqlalchemy import text

query = "\
SELECT p2.country, p2.continent, prime_minister, president \
FROM presidents AS p1 \
INNER JOIN prime_ministers AS p2 \
ON p1.country = p2.country \
LIMIT 5; "
df = pd.read_sql(text(query),eg)
df

,country,continent,prime_minister,president
0,Egypt,Africa,Mostafa Madbouly,Abdel Fattah el-Sisi
1,Portugal,Europe,António Costa,Marcelo Rebelo de Sousa
2,Pakistan,Asia,Shehbaz Sharif,Arif Alvi
3,India,Asia,Narendra Modi,Ram Nath Kovind


In [17]:
##########################################   SELF JOINING TABLES     #################################
from sqlalchemy import text

query = "\
SELECT p1.country AS Country1, p2.country AS Country2, p1.continent \
FROM prime_ministers AS p1 \
INNER JOIN prime_ministers AS p2 \
ON p1.continent = p2.continent \
  AND p1.country <> p2.country \
LIMIT 5;" 
# The AND statement here removes duplicates in the resulting table
df = pd.read_sql(text(query),eg)
df

,country1,country2,continent
0,Portugal,United Kingdom,Europe
1,Portugal,Norway,Europe
2,Pakistan,India,Asia
3,Pakistan,Brunei,Asia
4,Pakistan,Oman,Asia


In [18]:
from sqlalchemy import text

query = " \
SELECT p1.country_code, \
   p1.size AS size2010, \
   p2.size AS size2015 \
FROM populations AS p1 INNER JOIN populations AS p2 \
ON p1.country_code = p2.country_code \
WHERE p1.year = 2010 \
    AND p1.year = p2.year - 5;"

df = pd.read_sql(text(query),eg)
df
## Filter such that p1.year is always five years before p2.year WTF!!

,country_code,size2010,size2015
0,ABW,101597.0,103889.0
1,AFG,27962208.0,32526562.0
2,AGO,21219954.0,25021974.0
3,ALB,2913021.0,2889167.0
4,AND,84419.0,70473.0
...,...,...,...
212,XKX,1775680.0,1801800.0
213,YEM,23591972.0,26832216.0
214,ZAF,50979432.0,55011976.0
215,ZMB,13917439.0,16211767.0


In [19]:
###############################################        Introducing UNION Section 3         ####################################################
# NOTES
# JOINS merge tables on a particular criteria. ON code from our examples
# UNION stacks tables
# As I merge .csv file imported into postgres I will use UNION for similar datasets
# 




In [20]:
###################################        SECTION 4 Introducing Nested Queries: Sub Queries          #######################################

# Let's redo the last section on nested sub queirs
# These anti joins and semi joins are kind of unions and except functionality

#NOTES
 #Sub-queries can be used inside SELECT, FROM, or WHERE
 #Sub-queries can use a 3rd table as a filter criteria
 #Semi joins: chooses records in the left table where a condition is met in the right table
 #Anti joins: 

In [21]:
##################################################          NESTED IN WHERE       ############################################

# I'm using a nest query, sub-query in the WHERE clause to first filter which country achieved independence before 1800 from states table.
# Then from that list I'm selecting president, country, and continent from the presidents table. 

from sqlalchemy import text

query = "\
 SELECT president, country, continent \
 FROM presidents \
 WHERE country IN \
  (SELECT country \
  FROM states \
  WHERE indep_year < 1800);"
df = pd.read_sql(text(query),eg)
df

,president,country,continent
0,Marcelo Rebelo de Sousa,Portugal,Europe
1,Joe Biden,USA,North America


In [23]:
# ANTI JOIN
# Now I want to find out which countries did not achieve independence before 1800.
# Let's add an additional critera to limit our results to Americas

from sqlalchemy import text

query = "\
 SELECT president, country \
 FROM presidents \
 WHERE continent LIKE '%America' \
  AND country NOT IN \
   (SELECT country \
   FROM states \
   WHERE indep_year < 1800);"
df = pd.read_sql(text(query),eg)
df

,president,country
0,Luis Lacalle Pou,Uruguay
1,Gabriel Boric,Chile


In [26]:
from sqlalchemy import text

query = "\
SELECT code \
FROM countries \
WHERE region = 'Middle East';"
df = pd.read_sql(text(query),eg)
df

,code
0,ARE
1,ARM
2,AZE
3,BHR
4,GEO
5,IRQ
6,ISR
7,YEM
8,JOR
9,KWT


In [28]:
from sqlalchemy import text

query = "\
SELECT DISTINCT name \
FROM languages \
ORDER BY name ASC;"
df = pd.read_sql(text(query),eg)
df

,name
0,Afar
1,Afrikaans
2,Akyem
3,Albanian
4,Alsatian
...,...
391,Yapese
392,Yoruba
393,Yue
394,Zezuru


In [30]:
from sqlalchemy import text

query = "\
SELECT DISTINCT name \
FROM languages \
WHERE code IN \
 (SELECT code \
  FROM countries \
  WHERE region = 'Middle East') \
ORDER BY name;"
df = pd.read_sql(text(query),eg)
df

,name
0,Arabic
1,Aramaic
2,Armenian
3,Azerbaijani
4,Azeri
5,Baluchi
6,Bulgarian
7,Circassian
8,English
9,Farsi


In [32]:
#Give me a list of all the countries on the Oceania continent

from sqlalchemy import text

query = "\
SELECT code, name \
FROM countries \
WHERE continent = 'Oceania';"
df = pd.read_sql(text(query),eg)
df

,code,name
0,ASM,American Samoa
1,AUS,Australia
2,FJI,Fiji Islands
3,GUM,Guam
4,KIR,Kiribati
5,MHL,Marshall Islands
6,FSM,"Micronesia, Federated States of"
7,NRU,Nauru
8,PLW,Palau
9,PNG,Papua New Guinea


In [39]:
# Give me a list of currencies used in countries on the Oceania continent.
# Continent is not a field in the currencies table, but country code is.
# Country code is listed with continents on the countries table.
# INNER JOIN to bring in each of the fields from two tables into 1 table.

# SUMMARY: Give me a list of all the country's currencies used on the Oceania continent

from sqlalchemy import text

query = "\
SELECT c1.code, c1.name, basic_unit AS currency \
FROM countries AS c1 \
INNER JOIN currencies AS c2 \
 ON c1.code = c2.code \
WHERE c1.continent = 'Oceania';"

df = pd.read_sql(text(query),eg)
df

,code,name,currency
0,AUS,Australia,Australian dollar
1,PYF,French Polynesia,CFP franc
2,KIR,Kiribati,Australian dollar
3,MHL,Marshall Islands,United States dollar
4,NRU,Nauru,Australian dollar
5,NCL,New Caledonia,CFP franc
6,NZL,New Zealand,New Zealand dollar
7,PLW,Palau,United States dollar
8,PNG,Papua New Guinea,Papua New Guinean kina
9,WSM,Samoa,Samoan tala


In [47]:
# Now I want a list of all countries on the Oceania continent that are NOT in the currencies table
# Ok, this is going to compare the list of countries on the Oceania continent from the countries table against the list of countries on the currencies table.
# Then is going to create a list of countries on the Oceania continent that are NOT in the currency tables. (Maybe the country does not have a registered currency)

from sqlalchemy import text

query = "\
SELECT code, name \
FROM countries \
WHERE continent = 'Oceania' \
  AND code NOT IN \
    (SELECT code \
    FROM currencies);"
df = pd.read_sql(text(query),eg)
df

##################################### WTF!  THE 2 PREVIOUS QUESTIONS ARE WORDED HORRIBLY! ##########################

,code,name
0,ASM,American Samoa
1,FJI,Fiji Islands
2,GUM,Guam
3,FSM,"Micronesia, Federated States of"
4,MNP,Northern Mariana Islands


In [52]:
#
#################################################     NEST IN SELECT       #########################################


from sqlalchemy import text

query = "\
SELECT DISTINCT continent, \
    (SELECT COUNT (*) \
    FROM monarchs \
    WHERE states.continent = monarchs.continent) AS monarch_count \
FROM states;"
df = pd.read_sql(text(query),eg)
df

#############################################             FOUND CODING ERROR IN THE TRAINING MATERIAL  NO WONDER...        ##########################

,continent,monarch_count
0,Africa,0
1,Asia,2
2,Europe,2
3,North America,0
4,Oceania,0
5,South America,0


In [ ]:
#  Working from the same table

#from sqlalchemy import text

#query = "\
#SELECT * \
#FROM populations \
#WHERE life_expectancy > 1.15 * AVG(life_expectancy) \
#    AND year = 2015;"
#df = pd.read_sql(text(query),eg)
#df

####################################             this code doesn't even work in the training material          ###############################

In [63]:
from sqlalchemy import text

query = "\
SELECT * \
FROM populations \
WHERE year = 2015 \
 AND life_expectancy > 1.15 * \
   (SELECT AVG(life_expectancy) \
    FROM populations \
    WHERE year = 2015);"
df = pd.read_sql(text(query),eg)
df
###########################################      After all that, I forgot a )    :-( 

,pop_id,country_code,year,fertility_rate,life_expectancy,size
0,21,AUS,2015,1.833,82.451220,23789752.0
1,376,CHE,2015,1.540,83.197560,8281430.0
2,356,ESP,2015,1.320,83.380486,46443992.0
3,134,FRA,2015,2.010,82.670730,66538392.0
4,170,HKG,2015,1.195,84.278046,7305700.0
5,174,ISL,2015,1.930,82.860980,330815.0
6,190,ITA,2015,1.370,83.490240,60730584.0
7,194,JPN,2015,1.460,83.843660,126958470.0
8,340,SGP,2015,1.240,82.595120,5535002.0
9,374,SWE,2015,1.880,82.551216,9799186.0


In [72]:
from sqlalchemy import text

query = "\
SELECT cities.name, cities.country_code, cities.urbanarea_pop \
FROM cities \
INNER JOIN countries \
ON code = country_code \
WHERE cities.name = countries.capital \
ORDER BY urbanarea_pop DESC;"
df = pd.read_sql(text(query),eg)
df


,name,country_code,urbanarea_pop
0,Beijing,CHN,21516000.0
1,Dhaka,BGD,14543124.0
2,Tokyo,JPN,13513734.0
3,Moscow,RUS,12197596.0
4,Cairo,EGY,10230350.0
...,...,...,...
61,Dakar,SEN,1146053.0
62,Abu Dhabi,ARE,1145000.0
63,Tripoli,LBY,1126000.0
64,Yerevan,ARM,1060138.0


In [77]:
#INNER JOIN ONLY RETURNS PERFECT MATCHES FROM EACH TABLE
#LEFT JOIN KEEPS ALL RECORDS IN THE LEFT TABLE AND BRINGS IN MATCH DATA FROM THE RIGHT AND NULL FOR RECORDS THAT DO NOT MATCH

from sqlalchemy import text

query = "\
SELECT countries.name AS country, COUNT (*) AS cities_num \
FROM countries \
LEFT JOIN cities \
ON countries.code = cities.country_code \
GROUP BY countries.name \
ORDER BY cities_num DESC, country ASC \
LIMIT 9;"
df = pd.read_sql(text(query),eg)
df
############################        have to use GROUP BY with aggregate functions are in the SELECT statement like COUNT () SUM () AVG () MAX () MIN ()

,country,cities_num
0,China,36
1,India,18
2,Japan,11
3,Brazil,10
4,Pakistan,9
5,United States,9
6,Indonesia,7
7,Russian Federation,7
8,South Korea,7


In [89]:
from sqlalchemy import text

query = "\
SELECT countries.name AS country, \
   (SELECT COUNT (cities.name) \
    FROM cities \
    WHERE countries.code = cities.country_code) AS cities_num \
FROM countries \
ORDER BY cities_num DESC;"
df = pd.read_sql(text(query),eg)
df

#############################          In sub queries you do not need GROUP BY        ############################

,country,cities_num
0,China,36
1,India,18
2,Japan,11
3,Brazil,10
4,United States,9
...,...,...
200,Moldova,0
201,Monaco,0
202,Mongolia,0
203,Namibia,0


In [ ]:
#
#################################################     NEST IN FROM       #########################################
#

In [92]:
from sqlalchemy import text

query = "\
SELECT continent, MAX(indep_year) AS most_recent \
FROM states \
GROUP BY continent;"
df = pd.read_sql(text(query),eg)
df

,continent,most_recent
0,South America,1825
1,Oceania,1901
2,Africa,1922
3,Asia,1984
4,North America,1776
5,Europe,1905


In [94]:
from sqlalchemy import text

query = "\
SELECT DISTINCT monarchs.continent, sub.most_recent \
FROM monarchs, \
    (SELECT continent, MAX(indep_year) AS most_recent \
    FROM states \
    GROUP BY continent) AS sub \
WHERE monarchs.continent = sub.continent;"
df = pd.read_sql(text(query),eg)
df

,continent,most_recent
0,Asia,1984
1,Europe,1905


In [97]:
from sqlalchemy import text

query = "\
SELECT code, COUNT (name) as lang_num \
FROM languages \
GROUP BY code \
ORDER BY lang_num DESC;"
df = pd.read_sql(text(query),eg)
df

,code,lang_num
0,ZMB,19
1,ZWE,16
2,ETH,16
3,IND,14
4,NPL,14
...,...,...
207,COL,1
208,AIA,1
209,DOM,1
210,SAU,1


In [130]:
from sqlalchemy import text

query = "\
SELECT local_name, sub.lang_num \
FROM countries, \
    (SELECT code, COUNT(name) AS lang_num \
    FROM languages \
    GROUP BY code) AS sub \
WHERE countries.code = sub.code \
ORDER BY lang_num DESC;"
df = pd.read_sql(text(query),eg)
df

# COUNT(name) AS lang_num is a FIELD NAME in the table called sub   which is now  sub.lang_num

,local_name,lang_num
0,Zambia,19
1,YeItyop´iya,16
2,Zimbabwe,16
3,Nepal,14
4,Bharat/India,14
...,...,...
193,Cuba,1
194,Colombia,1
195,Nederland,1
196,Al-´Arabiya as-Sa´udiya,1


In [136]:
from sqlalchemy import text

query = "\
SELECT code, inflation_rate, unemployment_rate \
FROM economies \
WHERE year = 2015 \
 AND code IN \
     (SELECT code \
      FROM countries \
      WHERE gov_form LIKE '%Republic%' OR gov_form LIKE '%Monarchy%') \
ORDER BY inflation_rate;"
df = pd.read_sql(text(query),eg)
df

,code,inflation_rate,unemployment_rate
0,LBN,-3.749,NaN
1,ZWE,-2.410,NaN
2,KNA,-2.302,NaN
3,MHL,-2.169,NaN
4,VCT,-1.726,NaN
...,...,...,...
173,YEM,39.403,NaN
174,UKR,48.684,9.143
175,VEN,121.738,7.400
176,ARG,NaN,NaN


In [ ]:

-- Select fields from cities
SELECT 
	name, 
    country_code, 
    city_proper_pop, 
    metroarea_pop,
    city_proper_pop / metroarea_pop * 100 AS city_perc
FROM cities
-- Use subquery to filter city name
WHERE name IN
  (SELECT capital
   FROM countries
   WHERE (continent = 'Europe'
   OR continent LIKE '%America'))
-- Add filter condition such that metroarea_pop does not have null values
	  AND metroarea_pop IS NOT NULL
-- Sort and limit the result
ORDER BY city_perc DESC
LIMIT 10;